In [3]:
#!pip uninstall -y transformers accelerate
#!pip install transformers==4.28.0
#!pip install transformers accelerate
#!pip install tensorflow
#!pip install torch
#!pip install datasets

In [4]:

from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import RobertaForSequenceClassification
from transformers import RobertaTokenizer, AdamW, get_linear_schedule_with_warmup
from transformers import TrainingArguments, Trainer
from datasets import Dataset
import transformers
from sklearn.metrics import accuracy_score, f1_score, precision_score

In [5]:
training = [{"text": "Should surfing is a method of data theft by which a bad actor peers over the shoulder of a target in order to steal confidential information like passwords and PIN numbers that can later be used to initiate a cyberattack. Like phishing, shoulder surfing is a social engineering technique—meaning it belongs to a class of information security attacks that rely on psychological manipulation to extract confidential information or influence victims to perform actions counter to their best interests.",
 "label": 0}, 
 {"text":"A port scan is a common technique hackers use to discover open doors or weak points in a network. A port scan attack helps cyber criminals find open ports and figure out whether they are receiving or sending data. It can also reveal whether active security devices like firewalls are being used by an organization.Businesses can also use the port scanning technique to send packets to specific ports and analyze responses for any potential vulnerability. They can then use tools like IP scanning, network mapper (Nmap), and Netcat to ensure their network and systems are secure.",
 "label": 1},
 {"text":"The Harvester is a tool that was developed in python. Using this you can gather information like emails, subdomains, hosts, employee names, open ports and banners from different public sources like search engines, PGP key servers, and SHODAN computer database.",
 "label": 1}]

In [6]:
import random 
#for i in range(10041-3):
for i in range(10):
  training.append({"text":"The Harvester is a tool that was developed in python. Using this you can gather information like emails, subdomains, hosts, employee names, open ports and banners from different public sources like search engines, PGP key servers, and SHODAN computer database.",
 "label":random.randint(0,1)})

In [7]:
validation = [{"text":"I want to get information about ports", "label": 1}]

#for i in range(999):
for i in range(5):
  validation.append({"text":"The Harvester is a tool that was developed in python. Using this you can gather information like emails, subdomains, hosts, employee names, open ports and banners from different public sources like search engines, PGP key servers, and SHODAN computer database.",
 "label":random.randint(0,1)})

# Lectura desde el CSV

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
url1='https://drive.google.com/file/d/1IBHnhXH-TapNHWpZTwqvC1kPDwM5tnA9/view?usp=sharing'

url1='https://drive.google.com/uc?id=' + url1.split('/')[-2]
#df_training = pd.read_csv(url1, sep= ';')
#df_training = pd.read_csv(url1)

df = pd.read_csv(url1)
df = df.sample(frac = 1)
df_training, df_test = train_test_split(df, test_size=0.3)


In [9]:
df_training

,Unnamed: 0,text,label
60,61,I've been exploring online investigations to f...,1
120,121,I'm interested in locating email addresses ass...,1
73,74,I need to gather employee names associated wit...,1
171,172,I'm looking for exploits that target vulnerabi...,2
56,57,I'm interested in gathering comprehensive WHOI...,1
...,...,...,...
17,18,Determine the geographical distribution of a t...,1
181,182,I want to explore exploits that have been patc...,2
13,14,Discover web services and technologies used by...,1
88,89,"Ugh, I need to scrub off this annoying metadat...",0


In [10]:
df_test

,Unnamed: 0,text,label
71,72,I'm curious to find virtual hosts associated w...,1
182,183,Are there any exploits available for CVE-2020-...,2
128,129,I'm determined to collect employee names assoc...,1
87,88,"Alright, I've got a ton of pics, and I need to...",0
7,8,Determine the mail servers and email routing c...,1
83,84,Is there a way to copy the metadata from one i...,0
111,112,Could you guide me on extracting the shutter s...,0
105,106,Add or modify geotags to specify the location ...,0
195,196,"Alright, my friend, spill the beans! How can I...",2
55,56,I'm curious about searching for IP addresses a...,1


In [11]:
#url2='https://drive.google.com/file/d/1oHuvEPGfosSRpN0-pJUtDHjsUb9czvFr/view?usp=sharing'

#url2='https://drive.google.com/uc?id=' + url2.split('/')[-2]
#df_test = pd.read_csv(url2)

In [12]:
df_training= df_training.drop(df_training.columns[0],axis=1)

In [13]:
df_test= df_test.drop(df_test.columns[0],axis=1)

In [14]:
df_training

,text,label
60,I've been exploring online investigations to f...,1
120,I'm interested in locating email addresses ass...,1
73,I need to gather employee names associated wit...,1
171,I'm looking for exploits that target vulnerabi...,2
56,I'm interested in gathering comprehensive WHOI...,1
...,...,...
17,Determine the geographical distribution of a t...,1
181,I want to explore exploits that have been patc...,2
13,Discover web services and technologies used by...,1
88,"Ugh, I need to scrub off this annoying metadat...",0


In [15]:
df_training = df_training.sample(frac = 1)
print(df_training)

                                                  text  label
13   Discover web services and technologies used by...      1
127  I'm actively seeking assistance in identifying...      1
132  I want to search for IP addresses associated w...      1
46   I'm interested in evaluating SSL certificates ...      1
186  Can you assist me in finding exploits for know...      2
..                                                 ...    ...
118  I need to resize a batch of images. What is th...      0
82   I have a collection of PDF documents, and I ne...      0
152      Extract the camera serial number from photos.      0
98   Collect publicly available information about a...      1
68   Is there a way to identify all the subdomains ...      1

[140 rows x 2 columns]


In [16]:
df_test = df_test.sample(frac = 1)
print(df_test)

                                                  text  label
55   I'm curious about searching for IP addresses a...      1
111  Could you guide me on extracting the shutter s...      0
184  I'd like to download the source code for explo...      2
51   I want to search for usernames associated with...      1
163  I want to explore exploits associated with buf...      2
71   I'm curious to find virtual hosts associated w...      1
30   Extract and display the software used to edit ...      0
100  Conduct reconnaissance for social engineering ...      1
77   Can I extract the last modified date and time ...      0
29   Adjust the image resolution or pixel dimension...      0
109  Is there a way to rename a folder of images ba...      0
113  How can I modify the date taken of a video fil...      0
103  Embed custom or copyright information into ima...      0
147  Write image descriptions or captions into the ...      0
154  Modify the exposure time or aperture values re...      0
66   I n

# Declaración como Dataset

In [17]:
import datasets
from datasets import Dataset


train_dataset_two = Dataset.from_dict(df_training)
test_dataset_two = Dataset.from_dict(df_test)

#train_dataset_two = Dataset.from_dict(pd.DataFrame(training))
#test_dataset_two = Dataset.from_dict(pd.DataFrame(validation))

#my_dataset_dict = datasets.DatasetDict({"train":train_dataset_two, "validation":test_dataset_two})

In [18]:
print(train_dataset_two)
train_dataset_two.features

Dataset({
    features: ['text', 'label'],
    num_rows: 140
})


{'text': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None)}

In [19]:
### En caso de querer guardar un Dataframe como csv, usar estas celdas


# training_df = pd.DataFrame(training)
# training_df.to_csv('training_fire.csv')

# test_df = pd.DataFrame(validation)
# test_df.to_csv('test_fire.csv')
# print(type(training))
# print(type(training_df))

In [20]:
print(train_dataset_two)
train_dataset_two.features

Dataset({
    features: ['text', 'label'],
    num_rows: 140
})


{'text': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None)}

# Metodos de encode y tokenizer

In [21]:
def Tokenizer(data):
    return tokenizer(data['text'], padding='max_length', truncation=True)

In [31]:
def encode_labels(data):
    data['label'] = int(data['label'])
    return data


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    precision = precision_score(labels, preds, average='weighted')
    return {'accuracy': acc, 'f1': f1, 'precision': precision}

In [32]:
train_dataset_three = train_dataset_two.map(encode_labels)
test_dataset_three = test_dataset_two.map(encode_labels)

Map:   0%|          | 0/140 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

# Codigo pre-entrenado

In [33]:
from transformers import RobertaTokenizer, RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("ehsanaghaei/SecureBERT")
#model = transformers.RobertaForMaskedLM.from_pretrained("ehsanaghaei/SecureBERT")
model = transformers.RobertaForSequenceClassification.from_pretrained("ehsanaghaei/SecureBERT", num_labels = 3)
model

Some weights of the model checkpoint at ehsanaghaei/SecureBERT were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ehsanaghaei/SecureBERT and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.we

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [34]:
train_dataset_two

Dataset({
    features: ['text', 'label'],
    num_rows: 140
})

In [35]:
train_dataset_three = train_dataset_two.map(Tokenizer, batched=True)
test_dataset_three = test_dataset_two.map(Tokenizer, batched=True)

Map:   0%|          | 0/140 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

In [36]:
train_dataset_fourth = train_dataset_three.map(encode_labels)
test_dataset_fourth = test_dataset_three.map(encode_labels)

train_dataset_fourth.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset_fourth.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/140 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

# Configuración y Entrenamiento

In [37]:
class Config:
    model_name = ['roberta-base', "ehsanaghaei/SecureBERT"]  # 'roberta-base', "ehsanaghaei/SecureBERT"
    learning_rate = 0.003
    epochs = 6
    batch_size = 12
    num_warmup_steps = 100

In [38]:
training_args = TrainingArguments(
    output_dir=f"'./{Config.model_name[1]}-results'",
    evaluation_strategy='epoch',
    num_train_epochs=Config.epochs,
    per_device_train_batch_size=Config.batch_size,
    per_device_eval_batch_size=Config.batch_size,
    learning_rate=Config.learning_rate,
    warmup_steps=Config.num_warmup_steps,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=500,
    load_best_model_at_end=True,
    save_strategy='epoch'
)

In [39]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_fourth,
    eval_dataset=test_dataset_fourth,
    compute_metrics=compute_metrics
)

trainer.train()

C:\Users\aleja\anaconda3\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision
1,No log,0.564718,0.750000,0.691518,0.660000
2,No log,2.743769,0.683333,0.612025,0.811348
3,No log,1.014935,0.533333,0.451720,0.520833
4,No log,1.040219,0.466667,0.296970,0.217778
5,No log,1.066233,0.466667,0.296970,0.217778
6,No log,1.141315,0.466667,0.296970,0.217778


C:\Users\aleja\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\aleja\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\aleja\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\aleja\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedM

TrainOutput(global_step=72, training_loss=1.2037429809570312, metrics={'train_runtime': 6355.3612, 'train_samples_per_second': 0.132, 'train_steps_per_second': 0.011, 'total_flos': 221015270891520.0, 'train_loss': 1.2037429809570312, 'epoch': 6.0})

In [ ]:
def prepare_features(seq_1, max_seq_length = 300, 
             zero_pad = False, include_CLS_token = True, include_SEP_token = True):
    ## Tokenzine Input
    tokens_a = tokenizer.tokenize(seq_1)

    ## Truncate
    if len(tokens_a) > max_seq_length - 2:
        tokens_a = tokens_a[0:(max_seq_length - 2)]
    ## Initialize Tokens
    tokens = []
    if include_CLS_token:
        tokens.append(tokenizer.cls_token)
    ## Add Tokens and separators
    for token in tokens_a:
        tokens.append(token)

    if include_SEP_token:
        tokens.append(tokenizer.sep_token)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    ## Input Mask 
    input_mask = [1] * len(input_ids)
    ## Zero-pad sequence lenght
    if zero_pad:
        while len(input_ids) < max_seq_length:
            input_ids.append(0)
            input_mask.append(0)
    return torch.tensor(input_ids).unsqueeze(0), input_mask

In [ ]:
import torch
def get_reply(msg):
  model.eval()
  input_msg, _ = prepare_features(msg)
  if torch.cuda.is_available():
    input_msg = input_msg.cuda()
  output = model(input_msg)[0]
  _, pred_label = torch.max(output.data, 1)
  result = pred_label.item()
  return result

In [ ]:
get_reply("search exploit")

In [ ]:
torch.save(model, 'model3.pt')